In [ ]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
import os
import os.path as op
import sys
import itertools
from gensim.models import FastText
from gensim.matutils import softcossim
import argparse

# add path to utils module to python path
basename = r"" #path to basename file
path_utils = op.join(basename , "utils")
sys.path.insert(0, path_utils)

from sys_utils import load_library
load_library(op.join(basename, 'preprocess'))
load_library(op.join(basename, 'tweet_utils'))
load_library(op.join(basename, 'readWrite'))
os.environ["HADOOP_HOME"] = "/space/hadoop/hadoop_home"


from tweet_utils import *
from readWrite import readFile, savePandasDFtoFile
from preprocess import Preprocess
prep = Preprocess()


def cosinus_similarity(a, b):
    return np.inner(a,b)/(norm(a)*norm(b))
    #return np.dot(a, b.T)/(norm(a)*norm(b))


def delete_similar_tweets(df):
    if df.shape[0] == 1:
        return df
    else:
        
        all_indices = df.index.values.tolist()
        all_combinations = itertools.combinations(all_indices, 2)
        new_indices = []

        while(len(all_indices) > 1):
            first = all_indices[0]
            rest = all_indices[1::]
    
            tweet1 = prep.replace_hashtags_URL_USER(df.loc[first]["text"], mode_URL="replace", mode_Mentions="replace")
            vec1 = tweet_vectorizer(prep.tokenize(tweet1), model_ft)#.reshape(1,-1)
            for i in rest:
                tweet2 = prep.replace_hashtags_URL_USER(df.loc[i]["text"], mode_URL="replace", mode_Mentions="replace")
                vec2 = tweet_vectorizer(prep.tokenize(tweet2), model_ft)#.reshape(1,-1)

                cos = cosinus_similarity(vec1, vec2)
           
                if (cos > 0.98):
                    #print("1: ", df.loc[first]["text"])
                    #print("2: ", df.loc[i]["text"])
                    #print(cos)
                    #print("Remove", i, " :", df.loc[i]["text"])
                    all_indices.remove(i)
#            print("append", first, " : ", df.loc[first]["text"])
            new_indices.append(first)
            all_indices.remove(first)

        else:
            if len(all_indices) > 0:
 #               print("Append last", all_indices[0], " : ", df.loc[all_indices[0]]["text"])
                new_indices.append(all_indices[0])
 
        #print("\n New dataframe", df.ix[new_indices].shape)
        return df.ix[new_indices]


In [ ]:
tweets=pd.read_csv(r"") #import dataset

In [ ]:
delete_similar_tweets(tweets)

In [ ]:
tweets.to_csv(r"") #export dataset as csv